In [10]:
import pandas as pd
import numpy as np
import string
import re
from collections import Counter
from nltk.corpus import stopwords
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,  accuracy_score
import random 

from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.linear_model import LogisticRegression as LR
from sklearn import svm
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier as GB
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

import gensim #library needed for word2vec
from gensim.models import KeyedVectors

In [11]:
df = pd.read_csv('https://www.dropbox.com/s/ia4tpu47h5f5ptd/train.csv?raw=1')

In [104]:
df.sample(3)

def take_data(data):
    data = data.replace(np.nan, '', regex=True)
    return data['Plot']+data['Title']+data['Director']+data['Cast']#+data['Origin/Ethnicity']

In [111]:
nltk.download('stopwords')
nltk_stops = stopwords.words('english')
vectorizer = TfidfVectorizer(max_features=15000,
                             stop_words = nltk_stops,
                             max_df=.7)
data = take_data(df)
vectorizer.fit(data)
df_tf = vectorizer.transform(data)

[nltk_data] Downloading package stopwords to C:\Users\Andreas
[nltk_data]     Pap\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
df['Plot'].sample(10)

6153    Jessie is a middle-aged woman living with her ...
8309    At the beginning of the film, D.M.C. is releas...
3035    In 1980, an annual gathering of teams of idios...
6036    Paul Kersey (Charles Bronson) returns to New Y...
2722    After the death of his girlfriend, Suzuki (Iku...
9195    Recently widowed Michelle O'Brien moves into a...
1820    In 1988 San Dimas, California, slackers Bill a...
2904    "The film does not present a linear story; rat...
2254    The twin cities of Sodom and Gomorrah prosper ...
3398    San Francisco Assistant District Attorney Davi...
Name: Plot, dtype: object

In [117]:
# classifier = LR(max_iter=1000,class_weight='balanced' , solver='liblinear')
classifier = RF()#n_estimators= 1200, max_depth= 30, min_samples_split=5)
#classifier = svm.SVC(kernel='rbf', gamma=0.2)
#classifier = MLPClassifier(solver='lbfgs',alpha=1e-5, hidden_layer_sizes=(5, 2))
#classifier = GB(n_estimators=3000)

In [120]:

n_estimators = [100, 200, 500, 1000,1200]
max_depth = [2, 5, 10, 20, 30]
min_samples_split = [2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10] 

hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf)

gridF = GridSearchCV(classifier, hyperF, cv = 3, verbose = 1, 
                      n_jobs = -1)
bestF = gridF.fit(df_tf,df['Genre'])

test = pd.read_csv('https://www.dropbox.com/s/7mjx3ulzrf2pde5/test.csv?raw=1')
test_tf = vectorizer.transform(take_data(test))
test['Predicted'] = bestF.best_estimator_.predict(test_tf)
test[['Id','Predicted']].to_csv(f'test_submissionsCHECK{random.randint(50,5000)}.csv', index=False)

Fitting 3 folds for each of 80 candidates, totalling 240 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed: 16.2min
[Parallel(n_jobs=-1)]: Done 240 out of 240 | elapsed: 20.9min finished


In [ ]:
classifier = RF(n_estimators= 1200, max_depth= 30, min_samples_split=5)
classifier.fit(df_tf,df['Genre'])

test = pd.read_csv('https://www.dropbox.com/s/7mjx3ulzrf2pde5/test.csv?raw=1')
test_tf = vectorizer.transform(take_data(test))
test['Predicted'] = classifier(test_tf)
test[['Id','Predicted']].to_csv('test_submissionsRF.csv', index=False)

In [ ]:
classifier = LR(max_iter=1000,class_weight='balanced' , solver='liblinear')
classifier.fit(df_tf,df['Genre'])

test = pd.read_csv('https://www.dropbox.com/s/7mjx3ulzrf2pde5/test.csv?raw=1')
test_tf = vectorizer.transform(take_data(test))
test['Predicted'] = classifier(test_tf)
test[['Id','Predicted']].to_csv('test_submissionsLR.csv', index=False)

In [123]:
bestF.best_params_

{'max_depth': 30,
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'n_estimators': 1200}